<a href="https://colab.research.google.com/github/GiPaiva/Algoritmo-Genetico/blob/main/EP2_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes: Giovanna Paiva Alves e Matheus Sanchez Duda

# Equipamentos no Laboratório de Química do SENAC

Suponha que um laboratório de química precisa organizar o uso de seus equipamentos para garantir que todas as análises necessárias possam ser realizadas dentro do prazo esperado e que as restrições de segurança e capacidade dos equipamentos sejam cumpridas. O laboratório possui um conjunto de equipamentos, como balanças, espectrômetros e microscópios, que são usados para realizar diferentes tipos de análises químicas.

Existem várias análises que precisam ser realizadas, cada uma com seus próprios requisitos de equipamentos e tempo de execução. Alguns equipamentos só podem ser usados para uma análise específica, enquanto outros podem ser compartilhados entre diferentes análises.

As restrições a serem satisfeitas incluem:

1. Cada equipamento tem uma capacidade máxima de uso diário e só pode ser usado para uma análise por vez.
2. Uma análise não pode estar em 2 equipamentos ao mesmo tempo.
3. Cada análise fica 1 hora em cada equipamento

O objetivo é encontrar um cronograma de uso de equipamentos que atenda a todas as restrições e minimize o tempo total necessário para concluir todas as análises.


In [1]:
# Bibliotecas usadas
# Aleatorio
import random
# Dicionario
from collections import defaultdict
# Xerox
import copy

class Individuo:
    def __init__(self, analises, limites_equipamentos):
        # Analises e seus equipamentos necessários
        self.analises = analises

        # Equipamentos e seus limites diários de uso (em horas)
        self.limites_equipamentos = limites_equipamentos

        # Representação da agenda: dia -> hora -> [(equipamento, analise)]
        self.agenda = defaultdict(lambda: defaultdict(list))

        # Inicializa o rastreamento de uso dos equipamentos: dia -> equipamento -> horas usadas
        self.uso_equipamentos = defaultdict(lambda: defaultdict(int))

        # Validade da agenda
        self.agenda_valida = False

        # Inicializa com uma agenda aleatória
        self.inicializar_agenda_aleatoria()

    def inicializar_agenda_aleatoria(self):
        # Lista com todos os slots possiveis - cada dia da semana e cada hora do dia
        # 7 dias e das 8h até as 17h
        # 18h aparece, mas é por conta da pessoa que usa as 17h, pois cada equipamento pode ser usado 1h por cada analise
        todos_slots = [(dia, hora) for dia in range(1, 8) for hora in range(8, 18)]

        # Embaralha a lista de análises pra ficar bem aleatorio mesmo
        # É tipo tipo ver qual grupo vai ficar com qual tema de forma aleatoria
        lista_analises = list(self.analises.keys())
        random.shuffle(lista_analises)

        # Tenta agendar cada análise
        for analise in lista_analises:
            lista_equipamentos = self.analises[analise]

            # Pra cada equipamento que essa analise precisa usar
            for equipamento in lista_equipamentos:
                # Faz uma copia dos slots pra tentar alocar
                # Pra não bagunçar a lista original
                slots_disponiveis = todos_slots.copy()
                random.shuffle(slots_disponiveis)

                agendado = False
                for dia, hora in slots_disponiveis:
                    # Checa se esse horario é valido pra esse equipamento
                    # É tipo verificar se a sala de aula ta disponivel lá no lugar das chaves
                    if self.slot_valido(analise, dia, hora, equipamento):
                        self.agenda[dia][hora].append((equipamento, analise))
                        self.uso_equipamentos[dia][equipamento] += 1
                        agendado = True
                        break

                # Se não conseguiu agendar, o individuo fica invalido :(
                if not agendado:
                    self.agenda_valida = False
                    return

        # Se chegou até aqui é porque deu bom!
        # Tudo agendado certinho :D
        self.agenda_valida = True

    def slot_valido(self, analise, dia, hora, equipamento):
        """Verifica se um slot/tupla (dia, hora) tá livre pra usar esse equipamento com essa análise."""
        # Verifica se o equipamento já tá sendo usado nesse horário
        # Não dá pra usar a mesma balança ao mesmo tempo
        if any(equip == equipamento for equip, _ in self.agenda[dia][hora]):
            return False

        # Verifica se a análise já tá acontecendo nesse horário
        # Uma análise não pode estar em dois lugares ao mesmo tempo
        if any(anl == analise for _, anl in self.agenda[dia][hora]):
            return False

        # Verifica se o equipamento já bateu seu limite diário de uso
        if self.uso_equipamentos[dia][equipamento] >= self.limites_equipamentos[equipamento]:
            return False

        # Tudo certo, pode usar!
        return True

    def fitness(self):
        """Calcula o quão boa é essa agenda."""
        # Se a agenda não é valida, fitness é zero
        # é tipo tirar zero na prova porque esqueceu de assinar o nome hahaha (. ❛ ᴗ ❛.)
        if not self.agenda_valida:
            return 0.0

        # Conta quantas análises foram totalmente agendadas
        # É tipo contar os checks dos EPs concluidos
        completamente_agendadas = 0
        contador_analise_equipamento = defaultdict(int)

        for dia in range(1, 8):
            for hora in range(8, 18):
                for equipamento, analise in self.agenda[dia][hora]:
                    contador_analise_equipamento[(analise, equipamento)] += 1

        # Verifica se cada análise tem todos os equipamentos agendados
        # É como checar se todos os ingredientes da receita estão na mesa
        for analise, lista_equipamentos in self.analises.items():
            tudo_agendado = True
            for equipamento in lista_equipamentos:
                if contador_analise_equipamento[(analise, equipamento)] == 0:
                    tudo_agendado = False
                    break
            if tudo_agendado:
                completamente_agendadas += 1

        # Eficiência: relação entre o uso real e o uso máximo possível
        # É tipo calcular quanto % do tempo a gente realmente estudou
        uso_maximo_possivel = sum(self.limites_equipamentos.values()) * 7  # 7 dias
        uso_real = sum(sum(uso.values()) for uso in self.uso_equipamentos.values())
        eficiencia = uso_real / uso_maximo_possivel if uso_maximo_possivel > 0 else 0

        # Compactação: menos dias usados = agenda mais compacta
        # Melhor terminar tudo em 3 dias do que espalhar pela semana toda ne?
        total_dias_usados = len([dia for dia in range(1, 8) if any(self.agenda[dia].values())])
        compactacao = 1.0 / total_dias_usados if total_dias_usados > 0 else 0

        # Razão de completude - quanto % das analises realmente foi agendado
        # É como notas com virgula como numa prova com várias questões (será que dá para arredondar pra cima (●ˇvˇ●) )
        taxa_completude = completamente_agendadas / len(self.analises)

        # Junta tudo com pesos diferentes
        # Tipo media ponderada da escola
        fitness = (taxa_completude * 0.7) + (eficiencia * 0.2) + (compactacao * 0.1)
        return fitness

    def mutacao(self):
        """Cria uma cópia mutada deste individuo."""
        # Faz uma copia completa - tipo um clone
        mutado = copy.deepcopy(self)

        # Reseta tudo pra começar do zero
        mutado.agenda = defaultdict(lambda: defaultdict(list))
        mutado.uso_equipamentos = defaultdict(lambda: defaultdict(int))

        # Lista de todos os pares (analise, equipamento)
        # Tipo uma lista de todas as tarefas que precisam ser feitas
        todos_pares = []
        for analise, lista_equipamentos in self.analises.items():
            for equipamento in lista_equipamentos:
                todos_pares.append((analise, equipamento))

        # Embaralha tudo aleatoriamente
        random.shuffle(todos_pares)

        # Tenta reagendar cada par - agora com uma ordem diferente
        for analise, equipamento in todos_pares:
            todos_slots = [(dia, hora) for dia in range(1, 8) for hora in range(8, 18)]
            random.shuffle(todos_slots)

            agendado = False
            for dia, hora in todos_slots:
                if mutado.slot_valido(analise, dia, hora, equipamento):
                    mutado.agenda[dia][hora].append((equipamento, analise))
                    mutado.uso_equipamentos[dia][equipamento] += 1
                    agendado = True
                    break

            # Se não conseguiu encaixar, agenda inválida :(
            if not agendado:
                mutado.agenda_valida = False
                return mutado

        # Tudo deu certo! Agenda válida :D
        mutado.agenda_valida = True
        return mutado

    def crossover(self, outro):
        """Cria um filho combinando/cruzando os genes dos pais."""
        # Tipo misturar duas receitas pra ver se fica melhor
        # Ou como juntar os melhores genes dos pais
        filho = copy.deepcopy(self)

        # Limpa tudo pra começar de novo
        filho.agenda = defaultdict(lambda: defaultdict(list))
        filho.uso_equipamentos = defaultdict(lambda: defaultdict(int))

        # Lista todos os pares (analise, equipamento) que precisam ser agendados
        todos_pares = []
        for analise, lista_equipamentos in self.analises.items():
            for equipamento in lista_equipamentos:
                todos_pares.append((analise, equipamento))

        # Pra cada par, tenta pegar horários que funcionaram nos pais
        # É tipo juntar as melhores ideias de duas pessoas
        for analise, equipamento in todos_pares:
            slots_pai1 = []
            slots_pai2 = []

            # Procura onde cada pai agendou esse par
            for dia in range(1, 8):
                for hora in range(8, 18):
                    if (equipamento, analise) in self.agenda[dia][hora]:
                        slots_pai1.append((dia, hora))
                    if (equipamento, analise) in outro.agenda[dia][hora]:
                        slots_pai2.append((dia, hora))

            # Junta e embaralha os slots dos pais
            slots_potenciais = slots_pai1 + slots_pai2
            random.shuffle(slots_potenciais)

            agendado = False
            # Primeiro tenta os slots que funcionaram pros pais
            for dia, hora in slots_potenciais:
                if filho.slot_valido(analise, dia, hora, equipamento):
                    filho.agenda[dia][hora].append((equipamento, analise))
                    filho.uso_equipamentos[dia][equipamento] += 1
                    agendado = True
                    break

            # Se não deu certo, tenta qualquer slot
            # Plano B
            if not agendado:
                todos_slots = [(dia, hora) for dia in range(1, 8) for hora in range(8, 18)]
                random.shuffle(todos_slots)
                for dia, hora in todos_slots:
                    if filho.slot_valido(analise, dia, hora, equipamento):
                        filho.agenda[dia][hora].append((equipamento, analise))
                        filho.uso_equipamentos[dia][equipamento] += 1
                        agendado = True
                        break

            # Se mesmo assim não deu, o filho é inválido :(
            # As vezes a vida é assim mesmo, nem tudo funciona de primeira
            if not agendado:
                filho.agenda_valida = False
                return filho

        # Se chegou até aqui, deu tudo certo!
        # Temos um filho saudável :D
        filho.agenda_valida = True
        return filho

    def imprime(self):
        return f"Fitness: {self.fitness():.4f}, Válido: {self.agenda_valida}"

    def mostrar_agenda(self):
        print("Cronograma de Uso dos Equipamentos:")
        print("-" * 40)

        for dia in range(1, 8):
            print(f"\nDia {dia}:")
            print("---------")
            tem_atividade = False
            for hora in range(8, 18):
                if self.agenda[dia][hora]:
                    tem_atividade = True
                    print(f"{hora}:00 - {hora+1}:00:", end=" ")
                    atividades = []
                    for equipamento, analise in self.agenda[dia][hora]:
                        atividades.append(f"{equipamento} - {analise}")
                    print(", ".join(atividades))
            if not tem_atividade:
                print("Sem atividades agendadas")

        print("\nResumo de Uso dos Equipamentos por Dia:")
        for dia in range(1, 8):
            print(f"\nDia {dia}:")
            for equipamento in self.limites_equipamentos:
                uso = self.uso_equipamentos[dia][equipamento]
                print(f"  {equipamento}: {uso} horas (Limite: {self.limites_equipamentos[equipamento]} horas)")

    def mostrar_agenda_analises(self):
        print("\nCronograma por Análise:")
        print("-" * 40)
        agenda_analises = defaultdict(list)

        for dia in range(1, 8):
            for hora in range(8, 18):
                for equipamento, analise in self.agenda[dia][hora]:
                    agenda_analises[analise].append((dia, hora, equipamento))

        for analise in sorted(self.analises.keys()):
            print(f"\n{analise}:")
            if analise in agenda_analises:
                horarios = sorted(agenda_analises[analise], key=lambda x: (x[0], x[1]))
                for dia, hora, equipamento in horarios:
                    print(f"  Dia {dia}, {hora}:00 - {hora+1}:00: {equipamento}")
            else:
                print("  Não agendada")

class Populacao:
    def __init__(self, analises, limites_equipamentos, tamanho_populacao=20):
        # Inicializa a população com seus parâmetros básicos
        self.tamanho_populacao = tamanho_populacao
        self.analises = analises
        self.limites_equipamentos = limites_equipamentos
        self.populacao = []
        self.inicializacao()

    def inicializacao(self):
        # Cria os individuos iniciais
        for _ in range(self.tamanho_populacao):
            ind = Individuo(self.analises, self.limites_equipamentos)
            self.populacao.append(ind)
        # Ordena do melhor pro pior - quem tem o maior fitness para o menor
        self.populacao.sort(key=self._fitness_populacao, reverse=True)

    def mutacao(self):
        # Faz mutações em todos os indivíduos
        nova_lista = []
        for individuo in self.populacao:
            nova_lista.append(individuo.mutacao())
        return nova_lista

    def crossover(self):
        # Faz crossovers entre individuos da população
        # É tipo pai e mãe, juntando os cromossomos e tadãh, um filho, ou a junção/cruzamento dos dois
        nova_lista = []
        num_crossovers = self.tamanho_populacao

        for _ in range(num_crossovers):
            pai1 = self._selecao_torneio(3)
            pai2 = self._selecao_torneio(3)
            filho = pai1.crossover(pai2)
            nova_lista.append(filho)

        return nova_lista

    def _selecao_torneio(self, tamanho_torneio):
        # Seleciona indivíduos por torneio
        # Tipo escolher pessoas aleatorias para aprensentar o trabalho na lousa
        torneio = random.sample(self.populacao, tamanho_torneio)
        return max(torneio, key=self._fitness_populacao)

    def selecionar(self, populacao_mutada=None, populacao_cruzada=None):
        # Seleciona os melhores indivíduos para a próxima geração
        # É tipo a peneirada que acontece cada semestre
        candidatos = list(self.populacao)
        if populacao_mutada:
            candidatos.extend(populacao_mutada)
        if populacao_cruzada:
            candidatos.extend(populacao_cruzada)
        # Seleciona os melhores - só a nata da nata sobrevive!
        nova_lista = sorted(candidatos, key=self._fitness_populacao, reverse=True)
        self.populacao = nova_lista[:self.tamanho_populacao]

    def melhor_fitness(self):
        return self.melhor_individuo().fitness()

    def melhor_individuo(self):
        return self.populacao[0]

    def _fitness_populacao(self, individuo):
        return individuo.fitness()

class AlgoritmoGenetico:
    def __init__(self, populacao):
        self.populacao = populacao
        # Inicializa o erro com base no melhor fitness da população
        # Quanto menor o erro, mais perto estamos de uma solução
        self.erro = 1 - self.populacao.melhor_fitness()
        self.geracoes = 1

    def erro_final(self):
        return self.erro

    def qtd_geracoes(self):
        # #GeraçãoZ
        return self.geracoes

    def rodar(self, max_geracoes=1000, imprimir_em_geracoes=100, erro_min=0.01):
        # Executa o algoritmo genético até atingir o critério de parada

        # Atualiza o erro com base no melhor individuo atual
        self.erro = 1 - self.populacao.melhor_fitness()
        print(f"Geração: {self.geracoes}, Erro: {round(self.erro, 3)}, {self.populacao.melhor_individuo().imprime()}")

        while self.geracoes < max_geracoes and self.erro > erro_min:
            # Faz muutação e crossover
            # Deixa a seleção natural fazer seu trabalho
            populacao_mutada = self.populacao.mutacao()
            populacao_cruzada = self.populacao.crossover()

            # Seleciona os melhores pra próxima geração
            # Só os fortes sobrevivem! (ou os que tem melhor agenda, nesse caso)
            self.populacao.selecionar(populacao_mutada, populacao_cruzada)
            fitness = self.populacao.melhor_fitness()

            # Atualiza o erro - quanto mais perto de 1 o fitness, menor o erro
            # É tipo ver quanto falta pra passar na matéria 'v'
            self.erro = 1 - fitness

            self.geracoes += 1
            if self.geracoes % imprimir_em_geracoes == 0:
                print(f"Geração: {self.geracoes}, Erro: {round(self.erro, 3)}, {self.populacao.melhor_individuo().imprime()}")

        print(f"Geração: {self.geracoes}, Erro: {self.erro}, {self.populacao.melhor_individuo().imprime()}")
        return self.populacao.melhor_individuo()

In [2]:
# Entrada
# Lista de todos os equipamentos do laboratório e seus limites diários
limites = {
    "Balança Analítica": 6,
    "Agitador Magnético": 4,
    "Cromatógrafo Líquido": 8,
    "Cromatógrafo Gasoso": 6,
    "Espectrofotômetro UV-VIS": 4,
    "Espectrômetro Infravermelho": 6,
    "Espectrômetro de Massa": 4,
    "Microscópio": 6
}

# Lista de analises e quais equipamentos cada uma precisa
# Cada analise tem seu proprio "kit" de equipamentos necessários
analises = {
    "Análise 1": ["Espectrofotômetro UV-VIS", "Cromatógrafo Gasoso"],
    "Análise 2": ["Cromatógrafo Líquido", "Espectrômetro Infravermelho"],
    "Análise 3": ["Microscópio", "Balança Analítica"],
    "Análise 4": ["Espectrômetro de Massa"],
    "Análise 5": ["Agitador Magnético", "Espectrômetro Infravermelho"],
    "Análise 6": ["Cromatógrafo Líquido", "Espectrofotômetro UV-VIS"],
    "Análise 7": ["Espectrofotômetro UV-VIS", "Microscópio"],
    "Análise 8": ["Cromatógrafo Gasoso"],
    "Análise 9": ["Espectrômetro Infravermelho", "Balança Analítica"],
    "Análise 10": ["Espectrômetro de Massa", "Cromatógrafo Gasoso"]
}

# Inicializa a população com tamanho desejado
# Reunir e desenhar a arvore genealogica da familia
populacao = Populacao(analises, limites, tamanho_populacao=50)
genetica = AlgoritmoGenetico(populacao)

# Executa o algoritmo genético
# É tipo quando Deus deu play no mundo e viu a evolução acontecer
melhor_individuo = genetica.rodar(max_geracoes=500, imprimir_em_geracoes=50, erro_min=0.001)

print("\n\nMelhor Solução Encontrada:")
print(f"Fitness: {melhor_individuo.fitness()}")
print(f"É um indivíduo Válido? {melhor_individuo.agenda_valida}")

# Mostra os resultados finais
melhor_individuo.mostrar_agenda()
melhor_individuo.mostrar_agenda_analises()

Geração: 1, Erro: 0.268, Fitness: 0.7317, Válido: True
Geração: 50, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 100, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 150, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 200, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 250, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 300, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 350, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 400, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 450, Erro: 0.263, Fitness: 0.7367, Válido: True
Geração: 500, Erro: 0.255, Fitness: 0.7450, Válido: True
Geração: 500, Erro: 0.2549783549783551, Fitness: 0.7450, Válido: True


Melhor Solução Encontrada:
Fitness: 0.7450216450216449
É um indivíduo Válido? True
Cronograma de Uso dos Equipamentos:
----------------------------------------

Dia 1:
---------
Sem atividades agendadas

Dia 2:
---------
8:00 - 9:00: Espectrofotômetro UV-VIS - Análise 7
12:00 - 13:00: Espectrofotômetro 